In [ ]:
import scvelo as scv
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import scAnalysis as scrna
import scanpy as sc
import seaborn as sns

import warnings
warnings.filterwarnings("ignore")

scv.__version__
scv.settings.presenter_view = True  # set max width size for presenter view
scv.settings.set_figure_params('scvelo')  # for beautified visualization

In [ ]:
def do_pca(dfin,features,Npca=10,number_genes=[],zscore=True):
    #number_genes is the number of genes used to compute PCA, ordered bu std
    
    
    
    if number_genes==[]:
        features2=features
    else:
        number_genes=min(number_genes,len(features))
        df=dfin.copy()
        df.loc['stds',:]=[0]*len(df.columns)
        df.loc['stds',features] = df.loc[:,features].std(axis=0,ddof=0)/df.loc[:,features].mean(axis=0)
        df=df.sort_values('stds',axis=1,ascending=False)
        features2 = df.columns[0:number_genes]
        
    from sklearn.decomposition import PCA
    

    
    #Compte actual PCs using all samples
    print('Computing PCs...')
    X = dfin.loc[:,features2].dropna()
    if zscore:
        for col in X.columns:
            X[col] = (X[col] - X[col].mean())/X[col].std(ddof=0)
    X=X.fillna(0)
    pca = PCA(n_components=Npca)
    PCs = pca.fit_transform(X)
    perc = 100*pca.explained_variance_ratio_.sum()
    print('Done!')

    print('We use '+str(Npca)+' components to explain '+str(perc)+'% of the variability')

    #create DF with PCA results 
    dfout = dfin.copy()
    pclist = []
    for i in range(PCs.shape[1]):
        dfout.loc[:,'PC'+str(i+1)] = PCs[:,i]
        pclist.append('PC'+str(i+1))

    print(pca.explained_variance_ratio_)  
    print(pca.explained_variance_ratio_.sum())
    
    loadings = pca.components_.T * np.sqrt(pca.explained_variance_)
    loads = pd.DataFrame(loadings,index=features2,columns = pclist)
    loads =loads.sort_values('PC1',ascending=False)
    
    return dfout, loads

In [ ]:
import warnings
warnings.filterwarnings("ignore")

In [ ]:
#LOAD DATA
results_file = 'PC9.h5ad'
#results_file = 'HCC827velo_nolayer.h5ad'#'PC9.h5ad'#
#results_file = 'HCC4006.h5ad'
adata = sc.read('/home/estraja4/BFX_research/Resistance/barcoding/final_anndatasNEW/'+results_file)
celltype='PC9'

In [ ]:
adata

In [ ]:
scrna.plot_label(adata, feat='UMAP', stratify='louvain', legend_inside=True, figsize=(9, 7), ax='',savepdf=False)
plt.show()

In [ ]:
scrna.plot_label(adata, feat='UMAP', stratify='timepoint', legend_inside=False, figsize=(9, 7), ax='',savepdf=False)
plt.show()

In [ ]:
def add_kmeans(df,n_clusters=4):

    import sklearn.cluster as cluster
    import seaborn as sns

    features=[i for i in df.columns if 'PC' in i]
    X = df.loc[:,features]

    clustering = cluster.KMeans(n_clusters=n_clusters)#.AffinityPropagation()#n_clusters=n_clusters)#
    clustering.fit(X)

    df2=df.copy()
    df2['clustering']=[str(i) for i in clustering.labels_]
    
    #sns.lmplot(data=df2,x='PC1',y='PC2',hue='clustering',fit_reg=False)
    #for i in range(len(df)):
    #    plt.text(df2.iloc[i]['PC1'],df2.iloc[i]['PC2'],str(i))
    #plt.show()
    
    return df2

# Transitions based on cloneid
For each cloneid, compute its proportion in each of the louvain clusters. Use that information to compute likely transitions between timepoints.

In [ ]:
adata

In [ ]:
embedding='pca'

columns=[t for t in adata.obs['timepoint'].unique()]
for t in adata.obs['timepoint'].unique():
    for j in range(adata.obsm['X_'+embedding].shape[1]):
        columns.append(t+'_'+str(j))
    columns.append(t+'_count')
        
transitions=pd.DataFrame(index=[bc for bc in adata.obs['cloneid'].unique() if 'no-bc' not in bc and '-' not in bc],columns=columns)
for bc in adata.obs['cloneid'].unique():
    if 'no-bc' not in bc and '-' not in bc:
        aux=adata[adata.obs['cloneid']==bc]
        for t in adata.obs['timepoint'].unique():
            aux2=aux[aux.obs['timepoint']==t]
            val=np.median(aux2.obsm['X_'+embedding],axis=0)
            val=[i for i in val]
            transitions[t].loc[bc]=val
            for j in range(aux2.obsm['X_'+embedding].shape[1]):
                transitions[t+'_'+str(j)].loc[bc]=val[j]
            transitions[t+'_count'].loc[bc]=len(aux2)
transitions2pca=transitions.replace('nan',np.nan).dropna()

feats=[t for t in transitions2pca.columns if '_' in t and 'count' not in t]
transitions3pca,load = do_pca(transitions2pca,feats,Npca=4,number_genes=[],zscore=False)
transitions3pca=add_kmeans(transitions3pca,n_clusters=5)

In [ ]:
embedding='umap'

columns=[t for t in adata.obs['timepoint'].unique()]
for t in adata.obs['timepoint'].unique():
    for j in range(adata.obsm['X_'+embedding].shape[1]):
        columns.append(t+'_'+str(j))
    columns.append(t+'_count')
        
transitions=pd.DataFrame(index=[bc for bc in adata.obs['cloneid'].unique() if 'no-bc' not in bc and '-' not in bc],columns=columns)
for bc in adata.obs['cloneid'].unique():
    if 'no-bc' not in bc and '-' not in bc:
        aux=adata[adata.obs['cloneid']==bc]
        for t in adata.obs['timepoint'].unique():
            aux2=aux[aux.obs['timepoint']==t]
            val=np.median(aux2.obsm['X_'+embedding],axis=0)
            val=[i for i in val]
            transitions[t].loc[bc]=val
            for j in range(aux2.obsm['X_'+embedding].shape[1]):
                transitions[t+'_'+str(j)].loc[bc]=val[j]
            transitions[t+'_count'].loc[bc]=len(aux2)
transitions2umap=transitions.replace('nan',np.nan).dropna()

feats=[t for t in transitions2umap.columns if '_' in t and 'count' not in t]
transitions3umap,load = do_pca(transitions2umap,feats,Npca=4,number_genes=[],zscore=True)
#transitions3umap=add_kmeans(transitions3umap,n_clusters=5)

#read clustering from andata
df=scrna.andata2df(adata)
df=df[['cloneid','trajectory_class']]
mask1=~np.isin(df['trajectory_class'],['-'])
mask2=['earlier' not in i and 'later' not in i for i in df['trajectory_class']]
df=df.loc[mask1&mask2]

cid=[]
cl=[]
for c in df['cloneid'].unique():
    cid.append(c)
    cl.append(df.loc[df['cloneid']==c]['trajectory_class'][0])
df2=pd.DataFrame(index=cid)
df2['clustering']=cl
    
transitions3umap=transitions3umap.merge(df2,left_index=True,right_index=True)

In [ ]:

umap=transitions3umap[[i for i in transitions3umap.columns if 'PC' in i]+['clustering']]
umap=umap.rename(columns={i:i+'_UMAP' for i in umap.columns})
pca=transitions3pca[[i for i in transitions3pca.columns if 'PC' in i]+['clustering']]
pca=pca.rename(columns={i:i+'_PC' for i in pca.columns})

clusters = umap.merge(pca,left_index=True,right_index=True)

In [ ]:
clusters.head()

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(10, 5))

df=clusters
axs=ax[0]
hue='clustering_PC'
df=df.sort_values(hue)
for h in df[hue].unique():
    df2=df.loc[df[hue]==h]
    axs.scatter(df2['PC1_PC'],df2['PC2_PC'],marker='o',s=100,label=h)
    axs.legend(fontsize=16)
    axs.axes.xaxis.set_ticks([])
    axs.axes.yaxis.set_ticks([])
    axs.set_xlabel('PC1',fontsize=16)
    axs.set_ylabel('PC2',fontsize=16)
axs.set_title('PCA based, PC clusters')
    
axs=ax[1]
hue='clustering_UMAP'
df=df.sort_values(hue)
for h in df[hue].unique():
    df2=df.loc[df[hue]==h]
    axs.scatter(df2['PC1_UMAP'],df2['PC2_UMAP'],marker='o',s=100,label=h)
    axs.legend(fontsize=16)
    axs.axes.xaxis.set_ticks([])
    axs.axes.yaxis.set_ticks([])
    axs.set_xlabel('PC1',fontsize=16)
    axs.set_ylabel('PC2',fontsize=16)
axs.set_title('UMAP based, UMAP clusters')

fig, ax = plt.subplots(1, 2, figsize=(10, 5))

df=clusters
axs=ax[0]
hue='clustering_UMAP'
df=df.sort_values(hue)
for h in df[hue].unique():
    df2=df.loc[df[hue]==h]
    axs.scatter(df2['PC1_PC'],df2['PC2_PC'],marker='o',s=100,label=h)
    axs.legend(fontsize=16)
    axs.axes.xaxis.set_ticks([])
    axs.axes.yaxis.set_ticks([])
    axs.set_xlabel('PC1',fontsize=16)
    axs.set_ylabel('PC2',fontsize=16)
axs.set_title('PCA based, UMAP clusters')
    
axs=ax[1]
hue='clustering_PC'
df=df.sort_values(hue)
for h in df[hue].unique():
    df2=df.loc[df[hue]==h]
    axs.scatter(df2['PC1_UMAP'],df2['PC2_UMAP'],marker='o',s=100,label=h)
    axs.legend(fontsize=16)
    axs.axes.xaxis.set_ticks([])
    axs.axes.yaxis.set_ticks([])
    axs.set_xlabel('PC1',fontsize=16)
    axs.set_ylabel('PC2',fontsize=16)
axs.set_title('UMAP based, PCA clusters')

In [ ]:
#rename clusters for comparison: THIS HAS TO BE TWEAKED EVERY RUN!
clusters['clustering_PC']=clusters['clustering_PC'].map({'0':'3','1':'4','2':'1','3':'2','4':'0'})

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(12, 5))

df=clusters
axs=ax[0]
hue='clustering_UMAP'
df=df.sort_values(hue)
for h in df[hue].unique():
    df2=df.loc[df[hue]==h]
    axs.scatter(df2['PC1_UMAP'],df2['PC2_UMAP'],marker='o',s=100,label=h)
    axs.legend(fontsize=16)
    axs.axes.xaxis.set_ticks([])
    axs.axes.yaxis.set_ticks([])
    axs.set_xlabel('PC1',fontsize=16)
    axs.set_ylabel('PC2',fontsize=16)
axs.set_title('UMAP clusters',fontsize=14)

    
axs=ax[1]
hue='clustering_PC'
df=df.sort_values(hue)
for h in df[hue].unique():
    df2=df.loc[df[hue]==h]
    axs.scatter(df2['PC1_UMAP'],df2['PC2_UMAP'],marker='o',s=100,label=h)
    axs.legend(fontsize=16)
    axs.axes.xaxis.set_ticks([])
    axs.axes.yaxis.set_ticks([])
    axs.set_xlabel('PC1',fontsize=16)
    axs.set_ylabel('PC2',fontsize=16)
axs.set_title('PCA clusters',fontsize=14)
plt.show()
fig.savefig("For_referees/clustersPCvsUAMP_MGHPC9.pdf", bbox_inches='tight')

In [ ]:
#clusters.to_csv('For_referees/clustersPCvsUAMP_HCC4006.csv')
clusters.to_csv('For_referees/clustersPCvsUAMP_PC9.csv')

In [ ]:
#Import libraries
from matplotlib_venn import venn2, venn2_circles, venn2_unweighted
from matplotlib_venn import venn3, venn3_circles
from matplotlib import pyplot as plt

fig, axs = plt.subplots(1, 5, figsize=(20, 5))
k=0
for c in ['0','1','2','3','4']:
    ax=axs[k]
    maskUMAP=clusters['clustering_UMAP']==c
    maskPC=clusters['clustering_PC']==c
    a = len(clusters.loc[maskUMAP&~maskPC])
    b = len(clusters.loc[maskPC&~maskUMAP])
    ab = len(clusters.loc[maskPC&maskUMAP])
    k=k+1

    venn2(subsets = (a, b, ab), set_labels = ('UMAP', 'PCA'),ax=ax)
    ax.set_title('trajectory class '+c)
plt.show()
fig.savefig("For_referees/vennPCvsUAMP_PC9.pdf", bbox_inches='tight')